## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

## Modelos y campos

In [2]:
MODELOS = {
    "sentence_similarity": 'hiiamsid/sentence_similarity_spanish_es',
    "sentence-transformers 1": 'uzairnaeem/finetuned_sentence_similarity_spanish_es',
    "sentence-transformers 2": "mrm8488/modernbert-embed-base-ft-sts-spanish-matryoshka-768-64-5e",
    "sentence-transformers 3" : 'Santp98/SBERT-pairs-bert-base-spanish-wwm-cased'
}

CAMPOS = ["Titulo", "resumen", "objetivos", "claves"]

df = pd.read_csv("protocolos_completo_limpios.csv")

## Comparaciones y creación de tabla de consultas

In [17]:
# Solicitar consulta al usuario
while True:
    query = input("Ingresa el texto para comparar: ").strip()
    if not query:
        print("Consulta vacía. Por favor, ingresa un texto.")
        continue
    print("\nConsulta: ", query)
    break

filas_tabla = []

# comparación con cada modelo
for clave_modelo, nombre_modelo in MODELOS.items():
    print(f"Procesando modelo: {clave_modelo}")
    modelo = SentenceTransformer(nombre_modelo)
    df_temp = df.copy()

    for campo in CAMPOS:
        nombre_pkl = f"{campo}_{clave_modelo}_embeddings.pkl"
        ruta_pkl = os.path.join("PKL", nombre_pkl)

        if not os.path.exists(ruta_pkl):
            print(f"No se encontró el archivo {ruta_pkl}")
            continue

        with open(ruta_pkl, "rb") as f:
            embeddings_cargados = pickle.load(f)

        embedding_query = modelo.encode(query, convert_to_tensor=False)
        sim_coseno = cosine_similarity([embedding_query], embeddings_cargados)[0]
        df_temp[f"similitud_{campo}"] = sim_coseno

    columnas_sim = [f"similitud_{c}" for c in CAMPOS if f"similitud_{c}" in df_temp.columns]
    if not columnas_sim:
        continue

    df_temp["similitud_total"] = df_temp[columnas_sim].sum(axis=1)
    df_top = df_temp.sort_values("similitud_total", ascending=False).head(5)

    for i, (_, fila) in enumerate(df_top.iterrows(), start=1):
        for campo in CAMPOS:
            texto = fila[campo]
            fila_resultado = {
                "Modelo": clave_modelo,
                "Consulta": query if campo == "Titulo" and i == 1 else "",
                "No. Resultado más parecido": i,
                "Campo": campo,
                "Texto": texto,
                "Similitud": fila.get(f"similitud_" + campo, 0),
                "Justificación": ""
            }
            filas_tabla.append(fila_resultado)

        filas_tabla.append({
            "Modelo": clave_modelo,
            "Consulta": "",
            "No. Resultado más parecido": i,
            "Campo": "TOTAL",
            "Texto": "",
            "Similitud": fila["similitud_total"],
            "Justificación": ""
        })

df_nuevos = pd.DataFrame(filas_tabla)
df_nuevos = df_nuevos.loc[:, ~df_nuevos.columns.duplicated()]
nombre_archivo = "tabla_consultas.csv"

if os.path.exists(nombre_archivo):
    df_existente = pd.read_csv(nombre_archivo)
    df_nuevos = df_nuevos[df_existente.columns]
    df_final = pd.concat([df_existente, df_nuevos], ignore_index=True)
else:
    df_final = df_nuevos

df_final.to_csv(nombre_archivo, index=False)


Consulta:  Influencia de la alimentación en el rendimiento deportivo
Procesando modelo: sentence_similarity
Procesando modelo: sentence-transformers 1
Procesando modelo: sentence-transformers 2
Procesando modelo: sentence-transformers 3
